In [ ]:
import cv2
import mediapipe as mp
import time
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands   = mpHands.Hands()
mpDraw  = mp.solutions.drawing_utils


devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

volRange = volume.GetVolumeRange()

minVol = volRange[0]
maxVol = volRange[1]
volBar = 400
vol    = 0
volPer = 0

while True:
    success,img = cap.read()
    
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results= hands.process(imgRGB)
    #print(results.multi_hand_landmarks)
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lmList = []
            for id,lm in enumerate(handLms.landmark):
                #print(id,lm)
                h,w,c = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                #print(id,cx,cy)
                lmList.append([id, cx, cy])
                
            #print(lmList)
            mpDraw.draw_landmarks(img,handLms,mpHands.HAND_CONNECTIONS)
            
            if lmList:
                x1, y1 = lmList[4][1] , lmList[4][2]
                x2, y2 = lmList[8][1] , lmList[8][2]
                cv2.circle(img,(x1,y1),7,(255,0,0),cv2.FILLED)
                cv2.circle(img,(x2,y2),7,(255,0,0),cv2.FILLED)
                cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
                
                length = math.hypot(x2-x1,y2-y1)
                #print(length)
                
                vol = np.interp(length,[10,130],[minVol,maxVol])
                volBar = np.interp(length,[10,130],[400,150])
                volPer = np.interp(length,[10,130],[0,100])
                print(vol)
                volume.SetMasterVolumeLevel(vol, None)
                
        cv2.rectangle(img,(50,150),(85,400),(255,0,0),3)
        cv2.rectangle(img,(50,int(volBar)),(85,400),(255,0,0),cv2.FILLED)
        cv2.putText(img,f'{int(volPer)}%',(40,450),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),3)

               
    cv2.imshow("Image",img)
    cv2.waitKey(1)
